Clean archive data

In [3]:
import pandas as pd
from tabula import read_pdf
import requests, bs4, time
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
from datetime import date
import pickle
import calendar

In [221]:
with open('C:/Users/lundr/economics/GDP_Forecasts/raw_data/all_data_archive.pkl','rb') as f:
    df_arc = pickle.load(f)

In [222]:
import numpy as np

def check_float(x):
    try:
        a = pd.to_numeric(x)
        if np.isnan(a):
            b = False
        else:
            b = isinstance(a,float)
    except:
        b = False
    return b

def check_star(x):
    try:
        if '*' in x:
            b = 1
        else:
            b=0
    except:
        b=0
    return b

def close_gaps(text):
    gaps = re.findall('[A-z]'+' '+'[a-z]', text)
    for g in gaps:
        ng = ''.join(g.split(' '))
        text = text.replace(g,ng)
    return text


In [223]:
df_arc.pop('July_1999', None)

{4: [Empty DataFrame
  Columns: [/LYHUSRRO 8QLY, -XO, F, G, H]
  Index: []],
 7: [Empty DataFrame
  Columns: [/LYHUSRRO 8QLY, -XO, F, G, H]
  Index: []]}

In [224]:
for k in df_arc:
    for n in df_arc[k].keys():
        for z in range(len(df_arc[k][n])):
            month = k.split('_')[0]
            year = k.split('_')[1]
            if n == np.min(list(df_arc[k].keys())):
                    add = 0
            else:
                    add = 1
            
            if month.capitalize() in calendar.month_name[2:13]:

                forecast_year = 'Forecast_{}'.format(pd.to_numeric(year)+add)
                df_arc[k][n][z]['yr_forecast'] = pd.to_numeric(year)+add
            
            else:
                forecast_year='Forecast_{}'.format(pd.to_numeric(year)+add-1)
                df_arc[k][n][z]['yr_forecast'] = pd.to_numeric(year)+add-1

            cols_ = list(df_arc[k][n][z].columns)
            cols_ = [cols_[-1]] + cols_[:-1]
            df_arc[k][n][z] = df_arc[k][n][z][cols_]


            for i in df_arc[k][n][z].columns[1:]:
                num = df_arc[k][n][z][i].map(lambda x : check_float(x)).sum()
                if num > 0:
                    df_arc[k][n][z].rename(columns = {i:forecast_year}, inplace = True)
                    break
                else:
                        pass

            cols = [col for col in  df_arc[k][n][z] if col.startswith('Forecast_')]
            df_arc[k][n][z]=df_arc[k][n][z].loc[:,:cols[0]].copy()
       

In [233]:
for k in df_arc:
    for n in df_arc[k].keys():
        for z in range(len(df_arc[k][n])):
            df_arc[k][n][z]['Forecaster_']=''
            df_arc[k][n][z]['Month_of_Forecast']=''
            df_arc[k][n][z]['forecast_publication']=k
            for col in df_arc[k][n][z].columns:
                # split months from forecasters
                if ('forecaster' in col.lower()) or ('city' in col.lower()) :
                    for i in range(df_arc[k][n][z].shape[0]):
                        for m in list(calendar.month_abbr[1:])+['M ar','J an','July','June']:
                            if m in str(df_arc[k][n][z][col][i]):
                                df_arc[k][n][z]['Forecaster_'][i] = df_arc[k][n][z][col][i].split(m)[0]
                                df_arc[k][n][z]['Month_of_Forecast'][i] = m                        
                            else:
                                df_arc[k][n][z]['Forecaster_'][i] = df_arc[k][n][z][col][i]
                              
                                
                if ('unnamed' in col.lower()) & any(x in list(df_arc[k][n][z][col]) for x in list(calendar.month_abbr)):
                     df_arc[k][n][z]['Month_of_Forecast'] = df_arc[k][n][z][col].copy()  
                for i in  range(df_arc[k][n][z]['Month_of_Forecast'].shape[0]):
                    df_arc[k][n][z]['Month_of_Forecast'][i] = str(df_arc[k][n][z]['Month_of_Forecast'][i]).replace('*','').replace(' ','')
                    if df_arc[k][n][z]['Month_of_Forecast'][i]=='nan':
                        df_arc[k][n][z]['Month_of_Forecast'][i]=np.nan

C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [234]:
m_dict = {'':'',
 'Apr':'Apr',
 'Aug':'Aug',
 'Dec':'Dec',
 'Feb':'Feb',
 'Jan':'Jan',
 'Jul':'Jul',
 'July':'Jul',
 'Jun':'Jun',
 'June':'Jun',
 'Mar':'Mar',
 'May':'May',
 'NA':'',
 'Nov':'Nov',
 'Oct':'Oct',
 'Sep':'Sep',
 'Sept':'Sep',
 'madeinlast3months':'',
 'nan':''}

for k in df_arc.keys():
    for  n in df_arc[k].keys():
        for z in range(len(df_arc[k][n])):
            for i in range(df_arc[k][n][z].shape[0]):
                   df_arc[k][n][z]['Month_of_Forecast'][i]=m_dict[str(df_arc[k][n][z]['Month_of_Forecast'][i])]

                   
                           
                
                
#                 for x in list(calendar.month_abbr):
#     if x.lower() in str(df_arc[k][n][z]['Month_of_Forecast'][i]).lower():
#             df_arc[k][n][z]['Month_of_Forecast'][i] = x
#                df_arc[k][n][z] = df_arc[k][n][z].dropna(subset=df_arc[k][n][z].columns[2:],how='all').reset_index(drop=True)


C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [235]:
for k in df_arc.keys():
    for  n in df_arc[k].keys():
        for z in range(len(df_arc[k][n])):
            df_arc[k][n][z]['yr_of_forecast'] = 0
            #df_all['mnth_of_forecast'] = 0
            m = k.split('_')[0]
            y = k.split('_')[1]
            for i in range(df_arc[k][n][z].shape[0]):
                try:
                    pos_m = list(calendar.month_name).index(m)
                    pos_M = list(calendar.month_abbr).index(df_arc[k][n][z]['Month_of_Forecast'][i])
                    if pos_M <= pos_m:
                        df_arc[k][n][z]['yr_of_forecast'][i] = y
                    else:
                        df_arc[k][n][z]['yr_of_forecast'][i] = int(y)-1
                except:
                    pass
            df_arc[k][n][z]['Forecaster_']=df_arc[k][n][z]['Forecaster_'].map(lambda x: close_gaps(str(x)))
            df_arc[k][n][z] = df_arc[k][n][z].dropna(subset=df_arc[k][n][z].columns[2:],how='all').reset_index(drop=True)
            col_select = [x for x in df_arc[k][n][z].columns if ('Forecast_' in x) or ('Forecaster_' in x) or ('Month_of_Forecast'in x) or ('forecast_publication'in x) or ('yr_of_forecast'in x) or ('yr_forecast' in x)]
            df_arc[k][n][z]= df_arc[k][n][z][col_select]

C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [212]:
import re
k='March_2000'
n=5
z=0
i=4

df_arc[k][n][z]


,yr_forecast,Forecast_2000,Forecaster,Month_of_Forecast,forecast_publication,yr_of_forecast
0,2000,3.2,Abn Amro,Jan,March_2000,2000
1,2000,3.0,Barclays Bank,Mar,March_2000,2000
2,2000,3.0,Barclays Capital,Mar,March_2000,2000
3,2000,3.4,Charterhouse,Mar,March_2000,2000
4,2000,2.6,Chase Manhattan,Mar,March_2000,2000
5,2000,2.8,Credit Lyonnais,Mar,March_2000,2000
6,2000,2.8,Credit Suisse First Boston,Feb,March_2000,2000
7,2000,3.2,Daiwa Instituteof Research,Feb,March_2000,2000
8,2000,3.3,Deutsche Bank,Mar,March_2000,2000
9,2000,3.0,Dresdner Kleinwort Benson,Mar,March_2000,2000


In [236]:
# concatenate all of the data frames
df_all = pd.DataFrame()
frames= [df_arc[k][n][z] for k in df_arc for n in df_arc[k].keys() for z in range(len(df_arc[k][n]))]
df_all = pd.concat(frames)
df_all.reset_index(drop=True, inplace=True)

In [237]:
df_all.columns

Index(['yr_forecast', 'Forecast_2010', 'Forecaster_', 'Month_of_Forecast',
       'forecast_publication', 'yr_of_forecast', 'Forecast_2011',
       'Forecast_2009', 'Forecast_2008', 'Forecast_2007', 'Forecast_2006',
       'Forecast_2005', 'Forecast_2004', 'Forecast_2003', 'Forecast_2002',
       'Forecast_2001', 'Forecast_2000', 'Forecast_1999', 'Forecast_1998',
       'Forecast_1997'],
      dtype='object')

In [238]:
screening_terms = ['(% of GDP)',
                   'Private consumption',
                   ' -',
                   'Domestic demand',
                   'GDP',
                   'Government consumption',
                   'Private Consumption',
                   'Total Exports',
                   'Fixed Investments',
                   'consumption',
                   'CPI (Q4)',
                   'Change in inventorie',
                   'contribution (% of ',
                   'potential GDP)',
                  'Total imports',
                   'Total exports',
                  'Fixed investment',
                   'potential )',
                   'M4 xIOFC Growth',
                   'Government ',
                   ' growth (ppt))',
                   '(ppt)' ,
                   'Net trade contribution',
                   ')',
                   '(£bn'
                  ]
for x in screening_terms: 
    for i in range(len(df_all['Forecaster_'])):
        df_all['Forecaster_'][i] = df_all['Forecaster_'][i].replace(x,'')
        df_all['Forecaster_'][i]=df_all['Forecaster_'][i].strip()

C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\lundr\Miniconda3\envs\dsj\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [232]:
df_all.Forecaster_.unique()

array(['Bankof America Merrill Lynch', 'Barclays Capital', 'BNP Paribas',
       'Capital Economics', 'Citigroup', 'Commerzbank', 'Credit Suisse',
       'Daiwa Capital', 'Deutsche Bank', 'Goldman Sachs', 'nan', 'HSBC',
       'ING Financial', 'J P Morgan', 'Lombard Street', 'Morgan Stanley',
       'RBC Capital', 'RBS Global Banking &',
       'Schroders Investment Management', 'Societe Generale',
       'Standard Chartered Bank', 'UBS', 'Non-Cityforecasters',
       'British Chambersof Commerce', 'Beacon Economic Forecasting',
       'Cambridge Econometrics', 'CBI', 'CEBR', 'Economic Perspectives',
       'Experian Business Strategies', 'EIU', 'IHS Global Insight',
       'ITEM Club', 'Liverpool Macro Research', 'NIESR',
       'Oxford Economics', 'EC', 'OECD', 'IMF',
       'Averageofforecastsmadeinthelast 3 months', 'Independent',
       'New (marked *', 'City', 'Rangeofforecastsmadeinthelast 3 months',
       'Highest', 'Lowest', 'Median', 'HM Treasury',
       'Daiwa Securities S

In [242]:
text = 'Highest 3 .0' 
dig = re.findall("\d+\.\d+", text)
dig

[]

In [ ]:
def close_gaps(text):
    gaps = re.findall('[A-z]'+' '+'[a-z]', text)
    for g in gaps:
        ng = ''.join(g.split(' '))
        text = text.replace(g,ng)
    return text
21`4312Q`2` asdef close_gaps(text):
    gaps = re.findall('[A-z]'+' '+'[a-z]', text)
    for g in gaps:
        ng = ''.join(g.split(' '))
        text = text.replace(g,ng)
    return text